In [14]:
import pandas as pd
import torch
from sklearn.utils import shuffle

In [15]:
import pandas as pd
rnames = ['userId', 'movieId', 'rating', "TimeStamp"]
ratings = pd.read_table("ml-1m.inter", header=0, names=rnames, engine='python')
ratings

,userId,movieId,rating,TimeStamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [16]:
ratings = shuffle(ratings)
ratio = 0.8
# ratings
train = ratings.copy()
test = ratings.copy()
train.iloc[int(ratio*len(ratings)):,2] = None
# train.iloc[int(ratio*len(ratings)):,'rating'] = 0
test.iloc[:int(ratio*len(ratings)),2] = None

In [17]:
ratings.describe()

,userId,movieId,rating,TimeStamp
count,1.000209e+06,1.000209e+06,1.000209e+06,1.000209e+06
mean,3.024512e+03,1.865540e+03,3.581564e+00,9.722437e+08
std,1.728413e+03,1.096041e+03,1.117102e+00,1.215256e+07
min,1.000000e+00,1.000000e+00,1.000000e+00,9.567039e+08
25%,1.506000e+03,1.030000e+03,3.000000e+00,9.653026e+08
50%,3.070000e+03,1.835000e+03,4.000000e+00,9.730180e+08
75%,4.476000e+03,2.770000e+03,4.000000e+00,9.752209e+08
max,6.040000e+03,3.952000e+03,5.000000e+00,1.046455e+09


In [18]:
rating_matrix = train.pivot(index='userId', columns='movieId', values='rating')
n_users, n_movies = rating_matrix.shape
# Scaling ratings to between 0 and 1, this helps our model by constraining predictions
min_rating, max_rating = train['rating'].min(), train['rating'].max()
rating_matrix = (rating_matrix - min_rating) / (max_rating - min_rating)
print(rating_matrix)
print(n_users*n_movies-rating_matrix.isnull().values.sum())

movieId  1     2     3     4     5     6     7     8     9     10    ...   
userId                                                               ...   
1         1.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...  \
2         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
3         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
4         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
5         NaN   NaN   NaN   NaN   NaN  0.25   NaN   NaN   NaN   NaN  ...   
...       ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
6036      NaN   NaN   NaN  0.25   NaN   NaN   NaN   NaN   NaN   NaN  ...   
6037      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
6038      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
6039      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
6040      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   

movieId  39

In [19]:
# Replacing missing ratings with -1 so we can filter them out later

rating_matrix[rating_matrix.isnull()] = -1
rating_matrix = torch.FloatTensor(rating_matrix.values)
print(rating_matrix)

tensor([[ 1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        ...,
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.]])


In [20]:
class Loss(torch.nn.Module):
    def __init__(self, lam_u=0.3, lam_v=0.3, lam_L=0.1):
        super().__init__()
        self.lam_u = lam_u
        self.lam_v = lam_v
        self.lam_L = lam_L

    def forward(self, matrix, u_features, v_features, aver_model):
        non_zero_mask = (matrix != -1).type(torch.FloatTensor)
        predicted = torch.sigmoid(torch.mm(u_features, v_features.t()))
        
        diff = (matrix - predicted)**2
        prediction_error = torch.sum(diff*non_zero_mask)

        # u_regularization = self.lam_u * torch.sum(u_features.norm(dim=1))
        # v_regularization = self.lam_v * torch.sum(v_features.norm(dim=1))

        u_regularization = self.lam_u * torch.sum(u_features**2)
        v_regularization = self.lam_v * torch.sum(v_features**2)
        L_regularization = self.lam_L * torch.sum((v_features - aver_model)**2)
        
        return prediction_error + u_regularization + v_regularization + L_regularization, prediction_error, L_regularization

In [31]:
# Federated Rec: average movie_features 

lr = 0.1
num_client = 200        # 100, 200
m = n_users//num_client
num_epoch = 100         # 100, 200
latent_vectors = 20     # 20, 30

# user_features = torch.randn(n_users, latent_vectors, requires_grad=True)

user_features = []
movie_features = []
for i in range(num_client):
    user_features.append(torch.randn(m, latent_vectors, requires_grad=True))
    movie_features.append(torch.randn(n_movies, latent_vectors, requires_grad=True))
with torch.no_grad():
    for i in range(num_client):
        user_features[i].data.mul_(0.01)
        movie_features[i].data.mul_(0.01)
RFRec_error = Loss(lam_u=0.1, lam_v=0, lam_L=10)       # lam_u=0.1, lam_v=0, lam_L=10

optimizer_client_set = []
optimizer_user_set = []
optimizer_movie_set = []

for i in range(num_client):
    optimizer_client = torch.optim.Adam([user_features[i], movie_features[i]], lr=lr) 
    optimizer_client_set.append(optimizer_client)
aver_movie_features = torch.randn(n_movies, latent_vectors).data.mul(0.01)
# aver_movie_features = torch.zeros(n_movies, latent_vectors)
for i in range(num_client):
    aver_movie_features += movie_features[i]/num_client
    # print('client %s'%i, movie_features[i])

error_list = []
previous = torch.zeros(n_movies, latent_vectors)
for step, epoch in enumerate(range(num_epoch)):
    aver_loss = 0
    aver_prediction_error = 0

    # local update
    tmp = torch.zeros(n_movies, latent_vectors)
    for i in range(num_client):
        optimizer_client_set[i].zero_grad()

        loss, prediction_error, L_regularization = RFRec_error(rating_matrix[i*m:(i+1)*m], user_features[i], movie_features[i], aver_movie_features)

        aver_loss += loss/num_client
        aver_prediction_error += prediction_error/num_client

        loss.backward(retain_graph=True)
        # for params in optimizer_client_set[i].param_groups:                      
        #     params['lr'] = 0.1 
        optimizer_client_set[i].step()
    
        # #laplace mechanism
        # noise = torch.distributions.laplace.Laplace(torch.tensor([0.0]), torch.tensor([0.1])).sample()
        # with torch.no_grad():
        #     tmp+= torch.clip(movie_features[i], min=-0.4, max=0.4) + noise

        # without perturbation
        tmp += movie_features[i]

    # server update
    aver_movie_features = tmp/num_client

    # # stop criterion
    # error = (torch.norm(aver_movie_features-previous,p=2))/torch.norm(aver_movie_features,p=2)
    # error_list.append(error.detach().numpy())
    # print(error.detach().numpy())
    # previous = aver_movie_features.clone()    

    if step % 10 == 0:
        # print(f"Step {step} loss, {aver_loss:.3f}")
        print(f"Step {step} prediction_error, {aver_prediction_error:.3f}")
        # print(L)
        # print(f"Step {step} loss, {loss:.3f}")

Step 0 prediction_error, 394.659
Step 10 prediction_error, 323.381
Step 20 prediction_error, 189.204
Step 30 prediction_error, 168.981
Step 40 prediction_error, 155.962
Step 50 prediction_error, 146.614
Step 60 prediction_error, 140.259
Step 70 prediction_error, 135.891
Step 80 prediction_error, 132.553
Step 90 prediction_error, 129.930


In [22]:
# import numpy as np
# import matplotlib.pyplot as plt
# np.save("error/ML-1m_RFRec_error.npy",error_list) 
# l = np.load("error/ML-1m_RFRec_error.npy") 
# print(l)
# plt.plot(l)
# plt.hlines(0.01, 0, 100, linestyles='dotted',color='#0081C9',alpha=1, linewidth=1)
# plt.ylim(0,0.2)

In [32]:
test_rating_matrix = test.pivot(index='userId', columns='movieId', values='rating')

test_rating_matrix[test_rating_matrix.isnull()] = -1
test_rating_matrix = torch.FloatTensor(test_rating_matrix.values)
test_rating_matrix
# print(test_rating_matrix.shape)

tensor([[-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        ...,
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [ 3., -1., -1.,  ..., -1., -1., -1.]])

In [34]:
non_zero_mask = (test_rating_matrix != -1).type(torch.FloatTensor)
num = torch.sum(non_zero_mask)
def Error(matrix, u_features, v_features):
    predicted_ratings = torch.sigmoid(torch.mm(u_features,v_features.t()))
    pred = (predicted_ratings*(max_rating - min_rating) + min_rating)*non_zero_mask[i*m:(i+1)*m]
    actual = matrix*non_zero_mask[i*m:(i+1)*m]
    AE_diff = torch.abs(pred - actual)
    SE_diff = (pred - actual)**2
    
    prediction_abs_error = torch.sum(AE_diff)
    prediction_squared_error = torch.sum(SE_diff)
    n_non_zero = torch.sum(non_zero_mask[i*m:(i+1)*m])
    return prediction_abs_error, prediction_squared_error, n_non_zero

AE_error = 0
SE_error = 0
num_non_zero = 0
movie_features = aver_movie_features
for i in range(num_client):
    AE_error += Error(test_rating_matrix[i*m:(i+1)*m], user_features[i], movie_features)[0]
    SE_error += Error(test_rating_matrix[i*m:(i+1)*m], user_features[i], movie_features)[1]
    num_non_zero += Error(test_rating_matrix[i*m:(i+1)*m], user_features[i], movie_features)[2]
test_MAE = AE_error/num_non_zero
test_RMSE = torch.sqrt(SE_error/num_non_zero)
print('test_MAE =', test_MAE.data.numpy())
print('test_RMSE =', test_RMSE.data.numpy())

test_MAE = 0.69232476
test_RMSE = 0.8862912
